<p style="font-size:18px; font-weight:bold;"> 2025 Olivia Debnath</p>
<p style="font-size:14px;">Dana-Farber Cancer Institute & Harvard Medical School</p>

In this notebook, we will analyze the raw Cell*Gene output files where we tabulated the expression of our protein of interest i.e, POI (e.g., ACSF3) & other metadata across cell types/states of different tissues.

As an input, Level-2 files (genenerated in Cell*Gene_geneExp_data_Level1_16042025_Step2.ipynb) are used for QC+filtering. 

This script is named "S0" (PPI_contextualization_CellxGene_S0_Final_17042025.ipynb) as it represents QC. 

For convenience, we restricted to the major tissues (e.g, brain, lungs, liver, heart, kidney) for which it will be possible to do orthogonal validations.

Our main pipeline for Step-0 (basic QC):

1. Load data files (CSV) per gene

2. Filter data

- Remove bulk-level data (Cell Type = "aggregated" or "cell")
- Remove small clusters (No of cells < 1000)
= Remove low-expression clusters (%Cells Expressing Gene < 10% OR below 25th percentile per tissue) => moved this to _S1 (Hypothesis 1 test)
- Remove genes with extremely low absolute expression (Expression < 0.1 in >90% of clusters)
- Compute the Z-score per tissue (Saved as a new column). However, normalized expression is the most useful for us.

3. Compute scRNA-score = 𝑊1×robustness + 𝑊2×expression specificity+ 𝑊3×cluster diversity

- W1 (Robustness) = 0.4 (slightly increased, as it indicates how widely a gene is expressed)
- W2 (Expression Specificity) = 0.4 (remains high, as specificity is important for biological function)
- W3 (Cluster Diversity) = 0.2 (reduced weight, as broad cluster distribution, might not always be informative)
scRNA-score=0.4×Robustness + 0.4× Expression Specificity + 0.2×Cluster Diversity
where, Robustness = %Cells Expressing Gene Expression specificity = Variance-normalized expression level per tissue Cluster diversity = Number of unique Cell Types (cell types/states) per gene per tissue (normalized by max across dataset)

Here, we reproduced all the findings from /PPI_contextualization_CellxGene_downstream_S0S1_26022025.ipynb (but earlier we just looked into POIs for which we had tested one benign & 1 pathogenic allele) - now it's run on all genes.

Please note that POIs (e.g., AC012254.2) which are not expressed in the single-cell datasets of Cell*Gene & hence excluded from this pipeline.



In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import os

The script will automatically detect all input files matching [POI]CELLxGENE_gene_expression (CSV file per POI enlisting PPIs)

- Ensures proper handling of missing columns with an explicit check.
- Fixes the z-score transformation function for correct scaling
- Add print statements for clarity.

In [2]:
print(os.path.exists("./results_11032025/Jess_PPI_21032025/PPI_preprocessed_15042025/Level2/")) 

True


In [3]:
#Define input directory
input_dir = "./results_11032025/Jess_PPI_21032025/PPI_preprocessed_15042025/Level2/"

#**Dynamically find all Excel files that match the *_PPI.xlsx pattern**
input_files = sorted([f for f in os.listdir(input_dir) if f.endswith("_PPI.xlsx")])

#**Full paths for each file**
input_paths = {file: os.path.join(input_dir, file) for file in input_files}

#**Print summary**
print("\n📂 Detected Gene-wise PPI Excel Files (Alphabetically sorted):")
for file in input_files:
    print(file)

print(f"\n✅ Total Gene-specific PPI files found: {len(input_files)}")



📂 Detected Gene-wise PPI Excel Files (Alphabetically sorted):
ACSF3_PPI.xlsx
ACTB_PPI.xlsx
ACY1_PPI.xlsx
ADIPOQ_PPI.xlsx
AGXT_PPI.xlsx
AHCY_PPI.xlsx
AIPL1_PPI.xlsx
ALAS2_PPI.xlsx
ALDOA_PPI.xlsx
ALOX5_PPI.xlsx
AMPD2_PPI.xlsx
ANKRD1_PPI.xlsx
ANXA11_PPI.xlsx
AP2S1_PPI.xlsx
APOA1_PPI.xlsx
APOD_PPI.xlsx
ASNS_PPI.xlsx
ATPAF2_PPI.xlsx
BAG3_PPI.xlsx
BANF1_PPI.xlsx
BCL10_PPI.xlsx
BFSP2_PPI.xlsx
BLK_PPI.xlsx
C1QA_PPI.xlsx
C1QB_PPI.xlsx
C1QC_PPI.xlsx
CA8_PPI.xlsx
CACNB4_PPI.xlsx
CCBE1_PPI.xlsx
CCDC103_PPI.xlsx
CD151_PPI.xlsx
CDA_PPI.xlsx
CDC73_PPI.xlsx
CDK4_PPI.xlsx
CDKN1A_PPI.xlsx
CDKN2C_PPI.xlsx
CFP_PPI.xlsx
CHN1_PPI.xlsx
CHRNG_PPI.xlsx
CLCNKA_PPI.xlsx
CLDN19_PPI.xlsx
CLN6_PPI.xlsx
COL10A1_PPI.xlsx
COL2A1_PPI.xlsx
COMP_PPI.xlsx
COQ8A_PPI.xlsx
CRADD_PPI.xlsx
CREB1_PPI.xlsx
CRYAA_PPI.xlsx
CRYAB_PPI.xlsx
CRYBB1_PPI.xlsx
CSNK1D_PPI.xlsx
CSNK1E_PPI.xlsx
CTH_PPI.xlsx
CTNNA3_PPI.xlsx
CUL3_PPI.xlsx
CXCL16_PPI.xlsx
CYP4F12_PPI.xlsx
DCX_PPI.xlsx
DDX20_PPI.xlsx
DIABLO_PPI.xlsx
DMC1_PPI.xlsx
DOLK_PPI.xlsx

In [4]:
#**Create output directory inside input_dir**
output_dir = os.path.join(input_dir, "filtered_S0")  #QC step-specific
os.makedirs(output_dir, exist_ok=True)

#**Define new naming format: replace full filename with new suffix**
output_paths = {
    file: os.path.join(output_dir, file.replace(".xlsx", "_filtered_S0_17042025.csv"))
    for file in input_files
}

In [5]:
#**Define Weights for scRNA-score computation**

W1 = 0.4  #Robustness (How widely a gene is expressed)
W2 = 0.4  #Expression Specificity (Tissue-level variation)
W3 = 0.2  #Cluster Diversity (Spread across cell types)

Understand scRNA score calculation:

1. Robustness (W1= 0.4):

- Definition: Measures how widely a gene is expressed across cells within a tissue.
- Computation: The fraction of cells expressing the gene (%Cells Expressing Gene).
- Why it matters: A gene that is expressed in many cells within a tissue is more biologically relevant than one expressed in just a few scattered cells.
            - Helps identify widely expressed genes with functional roles.
            - Genes with low robustness but high Expression Intensity may play specialized roles.


2. Expression Specificity (Cell type bias; W2 = 0.4)

- Definition: Measures how uniquely a gene is expressed across different cell types within a tissue.
- Computation: Normalized variation in expression across cell types within a tissue.
    - High Specificity = A gene is highly restricted to a single cell type.
    - Low Specificity = A gene is broadly expressed across multiple cell types.

- Why it matters: Genes uniquely expressed in specific cell types are more functionally specialized. Also, genes with low Robustness but high specificity may be critical for rare cell states.


3. Cluster Diversity (Spread across cell types/states; W3 = 0.2)

- Definition: Measures the number of different cell types a gene is expressed in within a tissue.
- Computation: The number of unique cell types a gene is expressed in, normalized across all tissues.
- Why it matters: A gene expressed in diverse cell types may have broader functional significance. Helps in identifying broadly expressed functional genes vs cell-type-restricted markers.


Another important consideration could be Tissue Breadth (Multi-tissue expression)

🔹 Is this gene expressed in just one tissue or multiple tissues?

🔹 Formula: Number of distinct tissues in which this gene is expressed, normalized across all genes.
    - High Tissue Breadth = A gene is expressed in many tissues.
    - Low Tissue Breadth = A gene is restricted to just one tissue.

🔹 Why it matters?

- High Tissue Breadth suggests a fundamental biological role across multiple systems.
- Low Tissue Breadth suggests a tissue-specific function.



Final scRNA score Calculation

To integrate all these features into a single metric:

scRNA-score = (0.4 × Robustness) + (0.4 × Expression Specificity) + (0.2 × Cluster Diversity)
            
🔹 Why are these assigned weights?

- Robustness, and Specificity are equally important.
- Cluster Diversity is weighted lower (since broad expression isn’t always functionally meaningful).

In [6]:
def process_file(df, protein_of_interest):
    """ Cleans, filters, normalizes, and computes scRNA-score for scRNA-seq data. """

    #**Fix column names: Strip spaces & rename correctly**
    df.columns = df.columns.str.strip()
    df.rename(columns={
        "Number of Cells Expressing Genes": "Num_Cells_Expressing_Gene",
        "%Cells Expressing Gene ": "%Cells Expressing Gene"  # optional, legacy case
    }, inplace=True)

    #**Print number of unique cell types before filtering**
    print(f"\n🔍 Before filtering: Unique Cell Types = {df['Cell Type'].nunique()}")

    #**Ensure necessary columns exist**
    required_columns = {"Tissue", "Cell Type", "Cell Count", "Gene Symbol", "Expression", "Num_Cells_Expressing_Gene"}
    if not required_columns.issubset(df.columns):
        print(f"❌ ERROR: Missing required columns in {protein_of_interest} data! Skipping...")
        print(f"Missing: {required_columns - set(df.columns)}")
        return None

    #**Check if the protein of interest is present**
    if protein_of_interest not in df["Gene Symbol"].astype(str).unique():
        print(f"⚠️ WARNING: {protein_of_interest} not found in Gene Symbol column! Skipping...")
        return None

    #**Recalculate %Cells Expressing Gene**
    df["%Cells Expressing Gene"] = (df["Num_Cells_Expressing_Gene"] / df["Cell Count"]) * 100

    #**Remove bulk-level data (aggregate, cell)**
    df = df[~df["Cell Type"].isin(["aggregated", "cell"])]

    #**Remove small clusters (#Cells < 1000)**
    df = df[df["Cell Count"] >= 1000]

    #**Ensure the POI is expressed above threshold in at least some cells**
    main_protein_present = df[(df["Gene Symbol"] == protein_of_interest) & (df["Expression"] > 0.1)]
    if main_protein_present.empty:
        print(f"⚠️ WARNING: {protein_of_interest} not detected above 0.1 in any valid cluster! Skipping file.")
        return None

    #**Keep only valid tissue-cell type pairs where POI is expressed >0.1**
    valid_tissue_cell_types = main_protein_present[["Tissue", "Cell Type"]].drop_duplicates()

    #**Merge to keep only relevant tissue-cell type pairs where POI is expressed >0.1**
    df = df.merge(valid_tissue_cell_types, on=["Tissue", "Cell Type"], how="inner")

    #**Remove genes with Expression < 0.1 in >90% of clusters**
    gene_low_expr = df.groupby("Gene Symbol")["Expression"].apply(lambda x: (x < 0.1).mean() > 0.9)
    low_expr_genes = gene_low_expr[gene_low_expr].index
    df = df[~df["Gene Symbol"].isin(low_expr_genes)]

    #**Compute Z-score per Tissue**
    df["Expression_Zscore"] = df.groupby("Tissue")["Expression"].transform(lambda x: zscore(x, nan_policy="omit"))

    #**Compute scRNA-score components**
    df["Robustness"] = df["%Cells Expressing Gene"]
    df["Expression_Specificity"] = df.groupby("Tissue")["Expression"].transform(
        lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() > x.min() else 0
    )
    df["Cluster_Diversity"] = df.groupby(["Tissue", "Gene Symbol"])["Cell Type"].transform("nunique")
    df["Cluster_Diversity"] /= df["Cluster_Diversity"].max()

    #**Compute scRNA-score**
    df["scRNA_score"] = (
        W1 * df["Robustness"] +
        W2 * df["Expression_Specificity"] +
        W3 * df["Cluster_Diversity"]
    )

    #**Print number of unique cell types after filtering**
    print(f"🔍 After filtering: Unique Cell Types = {df['Cell Type'].nunique()}")

    return df

In [24]:
# for file in input_paths:
#     print(f"Found file: {file}") #can identify all files 

In [7]:
#Pick one file to test: 
file = "./results_11032025/Jess_PPI_21032025/PPI_preprocessed_15042025/Level2/ACSF3_PPI.xlsx"
df = pd.read_excel(file)

# Check if POI exists
poi = "ACSF3"
print("Gene Symbol values:", df["Gene Symbol"].unique())
print("Looking for POI:", poi in df["Gene Symbol"].unique())

# Call your function
processed = process_file(df, poi)

if processed is not None:
    print("✅ QC passed:", processed.shape)
else:
    print("⚠️ QC failed or POI missing.")


Gene Symbol values: ['ACSF3' 'KRT40' 'RAB28' 'TRIM27']
Looking for POI: True

🔍 Before filtering: Unique Cell Types = 757
🔍 After filtering: Unique Cell Types = 613
✅ QC passed: (8536, 14)


In [8]:
#Check another file to test: 
file = "./results_11032025/Jess_PPI_21032025/PPI_preprocessed_15042025/Level2/STXBP1_PPI.xlsx"
df = pd.read_excel(file)

# Check if POI exists
poi = "STXBP1"
print("Gene Symbol values:", df["Gene Symbol"].unique())
print("Looking for POI:", poi in df["Gene Symbol"].unique())

# Call your function
processed = process_file(df, poi)

if processed is not None:
    print("✅ QC passed:", processed.shape)
else:
    print("⚠️ QC failed or POI missing.")

Gene Symbol values: ['STXBP1' 'CENPP' 'MAPK1' 'STX19' 'STX5' 'SYTL4' 'TRIM38']
Looking for POI: True

🔍 Before filtering: Unique Cell Types = 755
🔍 After filtering: Unique Cell Types = 597
✅ QC passed: (14245, 14)


In [9]:
#Now that it works on individual files, test it on all files: 

def process_all_files():
    """Processes all input Excel files and saves filtered results with proper logging."""

    #**STEP 1: Locate all input Excel files**
    input_files = sorted([
        f for f in os.listdir(input_dir)
        if f.endswith(".xlsx") and "_PPI" in f
    ])

    print("\n📋 Input Excel files found:")
    for f in input_files:
        print(" -", f)
    if not input_files:
        print("❌ No Excel files found in directory!")
        return

    #**STEP 2: Prepare file paths**
    input_paths = {f: os.path.join(input_dir, f) for f in input_files}
    output_dir = os.path.join(input_dir, "filtered_S0")
    os.makedirs(output_dir, exist_ok=True)
    output_paths = {
        f: os.path.join(output_dir, f.replace(".xlsx", "_filtered_S0_17042025.csv"))
        for f in input_files
    }

    #**STEP 3: Process each file**
    for file, input_path in input_paths.items():
        print(f"\n📂 Processing: {input_path}...")

        if not os.path.isfile(input_path):
            print(f"❌ File not found: {input_path}")
            continue

        protein_of_interest = file.split("_")[0]
        print(f"🧬 Protein of Interest: {protein_of_interest}")

        try:
            df = pd.read_excel(input_path)
            print(f"📊 Loaded: {df.shape}")
        except Exception as e:
            print(f"❌ Failed to read file: {e}")
            continue

        if "Gene Symbol" not in df.columns:
            print("❌ Missing 'Gene Symbol' column!")
            continue

        gene_list = df["Gene Symbol"].astype(str).unique()
        print(f"🧬 Found genes: {gene_list}")

        if protein_of_interest not in gene_list:
            print(f"⚠️ Skipping {file}: POI '{protein_of_interest}' not found in file.")
            continue

        processed_df = process_file(df, protein_of_interest)

        if processed_df is not None:
            output_path = output_paths[file]
            processed_df.to_csv(output_path, index=False)
            print(f"✅ Saved: {output_path} ({processed_df.shape[0]} rows)")
        else:
            print(f"⚠️ Skipped {file}: QC failed.")


In [10]:
#**Run all 242 files in the input directory** 
process_all_files()


📋 Input Excel files found:
 - ACSF3_PPI.xlsx
 - ACTB_PPI.xlsx
 - ACY1_PPI.xlsx
 - ADIPOQ_PPI.xlsx
 - AGXT_PPI.xlsx
 - AHCY_PPI.xlsx
 - AIPL1_PPI.xlsx
 - ALAS2_PPI.xlsx
 - ALDOA_PPI.xlsx
 - ALOX5_PPI.xlsx
 - AMPD2_PPI.xlsx
 - ANKRD1_PPI.xlsx
 - ANXA11_PPI.xlsx
 - AP2S1_PPI.xlsx
 - APOA1_PPI.xlsx
 - APOD_PPI.xlsx
 - ASNS_PPI.xlsx
 - ATPAF2_PPI.xlsx
 - BAG3_PPI.xlsx
 - BANF1_PPI.xlsx
 - BCL10_PPI.xlsx
 - BFSP2_PPI.xlsx
 - BLK_PPI.xlsx
 - C1QA_PPI.xlsx
 - C1QB_PPI.xlsx
 - C1QC_PPI.xlsx
 - CA8_PPI.xlsx
 - CACNB4_PPI.xlsx
 - CCBE1_PPI.xlsx
 - CCDC103_PPI.xlsx
 - CD151_PPI.xlsx
 - CDA_PPI.xlsx
 - CDC73_PPI.xlsx
 - CDK4_PPI.xlsx
 - CDKN1A_PPI.xlsx
 - CDKN2C_PPI.xlsx
 - CFP_PPI.xlsx
 - CHN1_PPI.xlsx
 - CHRNG_PPI.xlsx
 - CLCNKA_PPI.xlsx
 - CLDN19_PPI.xlsx
 - CLN6_PPI.xlsx
 - COL10A1_PPI.xlsx
 - COL2A1_PPI.xlsx
 - COMP_PPI.xlsx
 - COQ8A_PPI.xlsx
 - CRADD_PPI.xlsx
 - CREB1_PPI.xlsx
 - CRYAA_PPI.xlsx
 - CRYAB_PPI.xlsx
 - CRYBB1_PPI.xlsx
 - CSNK1D_PPI.xlsx
 - CSNK1E_PPI.xlsx
 - CTH_PPI.xlsx
 - CTNN

**Quick summary of what we did:**

🔹 **Step 1: Data cleaning & pre-processing** 
- Fix column names (strip spaces, standardize names)
- Ensure all necessary columns exist (Tissue, Cell Type, Gene Symbol, Expression, etc.)
- Recalculate %Cells Expressing Gene (Num_Cells_Expressing_Gene / Cell Count * 100)

🔹 **Step 2: Filtering based on POI (Protein of Interest)**
- Remove bulk-level data (Cell Type = "aggregated" or "cell")
- Remove small clusters (Cell Count < 1000)
- Ensure POI is expressed at a minimum threshold level (Expression > 0.1 in at least some cells)
- Drop cell types where POI is missing or has Expression ≤ 0.1 → ensures we don’t process interactions in irrelevant clusters
- Retain only tissue-cell type pairs where POI meets this threshold

🔹 **Step 3: Additional Gene-level filtering**
- Remove genes with extremely low absolute expression (Expression < 0.1 in >90% of clusters)
(We moved percentile-based filtering max(10%, 25th percentile) to Step 3 for hypothesis testing instead of prematurely removing weak signals.)

🔹 **Step 4: Compute Normalization & scores**
- Compute Expression Z-score (per tissue)
- Compute scRNA-score using:

W1 (Robustness) = % of cells expressing the gene
W2 (Expression Specificity) = How uniquely a gene is expressed per tissue
W3 (Cluster Diversity) = Spread of gene expression across different cell types

**Weights Adjusted**:

W1 = 0.4 → Slightly higher to prioritize robust expression
W2 = 0.4 → Maintained high importance for specificity
W3 = 0.2 → Lowered, as broad cluster distribution isn't always informative

Key fixes as compared to previous codes: 

✅ More accurate removal of weak/noisy POI signals (Expression > 0.1)
✅ Ensures POI must be present in a cell type/state before interactions are processed
✅ Retains interactions that are functionally relevant but may have low POI expression
✅ Avoids premature exclusion of weakly expressed yet functionally important genes (like MLH1)


In [11]:
#Define the output directory
filtered_dir = "./results_11032025/Jess_PPI_21032025/PPI_preprocessed_15042025/Level2/filtered_S0"

#Get all filtered files
filtered_files = [f for f in os.listdir(filtered_dir) if f.endswith("_filtered_S0_17042025.csv")]

#Print total number of output files
print(f"\n Total filtered output files: {len(filtered_files)}")


 Total filtered output files: 242


1. Structure Checks:
- Tissue + Cell Type structure present.
- Within a tissue+cluster combination, POI (e.g., ACSF3, LITAF) is present & ordered first. 


2. All values for:
- %Cells Expressing Gene 
- Robustness, Expression_Specificity, Cluster_Diversity & scRNA_score

3. Filtering/QC criteria:
- No cell type has <1000 cells
- No bulk-level aggregate/cell entries.
- No genes with expression <0.1 across 90% of cell types.
